In [6]:
# Import modules
import numpy as np

In [7]:
# Import data
d = np.fromfile('data/d.dat')
fP = np.fromfile('data/fP.dat')
fr = np.fromfile('data/fr.dat')
localK = np.fromfile('data/localK.dat')
solution = np.fromfile('data/solution.dat')